<font color='black'><font size="5"> QUESTO NOTEBOOK SI PREFIGGE L'OBIETTIVO DI ANALIZZARE LE PRESTAZIONI DEI GIOCATORI DI SERIE A IN TERMINI DI FANTAMEDIA NEGLI ULTIMI 5 ANNI (DALLA STAGIONE 2015/2016 ALLA STAGIONE 2021/2022), ESTRAENDO INSIGHT UTILI PER AFFRONTARE L'ASTA 2022/2023 </font>
***
    
I DATI SU CUI SI BASA L'ANALISI SONO OPEN E FORNITI DA FANTACALCIO.IT

In [1]:
# Import librerie necessarie
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import pandas_log
import dtale

In [2]:
# Import del dataset
stats_serie_a = pd.read_excel('Master_data_soccer_15_22.xlsx')

In [3]:
# Prima visualizzazione del dataset
stats_serie_a.head(5)

,Id,R,Nome,Squadra,Pv,Mv,Mf,Gf,Gs,Rp,...,R-,Ass,Amm,Esp,Au,Anno,QA,Perc_Increase_Mf,Perc_Increase_Gf,Almeno 15 pres ult stag
0,4,P,Sportiello,Atalanta,36,6.24,5.26,0,43,3,...,0,0,2,0,0,15,1,NaN,NaN,NO
1,4,P,Sportiello,Fiorentina,10,5.65,3.80,0,18,0,...,0,0,1,0,0,16,1,-0.277567,0.0,NO
2,4,P,Sportiello,Fiorentina,37,6.05,5.08,0,38,1,...,0,0,0,1,0,17,1,0.336842,0.0,NO
3,4,P,Sportiello,Frosinone,35,6.10,4.20,0,64,0,...,0,0,5,0,0,18,1,-0.173228,0.0,NO
4,4,P,Sportiello,Atalanta,6,6.17,5.33,0,5,0,...,0,0,0,0,0,19,1,0.269048,0.0,NO


All'interno del dataset, le colonne da "Id" a "QA" sono fornite direttamente da fantalcio, ad eccezione delle colonna "Anno" aggiunta per identificare a quale anno fa riferimento l'informazione.it mentre le successive sono state calcolate in excel. In particolare, 'Perc_Increase_Mf' è una colonna creata calcolando la percentuale di incremento/decremento della fantamedia a fine anno per ciascuno degli anni del periodo di osservazione (chiaramente sarà null il primo anno) mentre  'Perc_Increase_Gf' presenta il medesimo meccanismo ma in relazione ai gol fatti. Infine è stata aggiunta la colonna 'Almeno 15 pres ult stag' che indica se il giocatore ha preso voto in almeno 15 match disputati nell'ultima stagione. 

### ANALISI SU FANTAMEDIE STORICHE ###

In [4]:
# Calcolo dei primi 5 giocatori per fantamedia complessiva sui 5 anni
stats_serie_a.groupby('Nome') \
.agg(Media_Mf_overall=('Mf','mean')) \
.sort_values(by='Media_Mf_overall', ascending=False) \
.head(10)

,Media_Mf_overall
Nome,
Volpato,10.000000
Lukaku,8.610000
Immobile,8.327143
Ibrahimovic,8.000000
Bove,8.000000
Osimhen,7.750000
Pellegri,7.733333
Kean,7.685000
Zapata D.,7.677143


Possiamo vedere come compaiano alcuni giocatori che nel nostro campionato hanno fatto molto bene negli ultimi anni (Immobile, Ibra, Lukaku) mentre altri risultano particolarmente alti come media avendo giocato soltanto pochissime partite nelle quali hanno realizzato una grande prestazione. Dal nostro punto di vista, l'obiettivo è eliminare dall'analisi questi giocatori per poterci concentrare su quelli in grado davvero di generare valore

In [5]:
# Filtriamo a monte tutti i giocatori con meno di 15 presenze stagionali nell'ultima stagione di serie A
# Per completezza aggiungiamo anche il numero totale di match disputati
stats_serie_a[stats_serie_a['Almeno 15 pres ult stag']== 'SI'] \
.groupby('Nome') \
.agg(Media_Mf_overall=('Mf','mean'),Numero_totale_match = ('Pv','sum')) \
.sort_values(by='Media_Mf_overall', ascending=False) \
.head(10)

,Media_Mf_overall,Numero_totale_match
Nome,,
Immobile,8.327143,221
Ibrahimovic,8.000000,54
Osimhen,7.750000,51
Kean,7.685000,55
Zapata D.,7.677143,218
Dybala,7.672857,201
Abraham,7.620000,37
Muriel,7.556667,168
Dzeko,7.482857,228


Il dato risulta ora molto più interessante, anche se vediamo che alcuni calciatori (Abraham, ad esempio) hanno soltanto 1 anno di storia mentre altri presentano uno storico molto più ampio. Proviamo quindi a dividere l'analisi in giocatori con meno di 2 stagioni e giocatori con piu di due stagioni e vediamo separatamente le due classifiche

In [6]:
# Migliori fantamedie tra giocatori con meno di due anni di storico in Serie A
mf_giocatori_over_80 = stats_serie_a[stats_serie_a['Almeno 15 pres ult stag']== 'SI'] \
.groupby('Nome') \
.agg(Media_Mf_overall=('Mf','mean'),Numero_totale_match = ('Pv','sum')) \
.sort_values(by='Media_Mf_overall', ascending=False)

mf_giocatori_over_80[mf_giocatori_over_80['Numero_totale_match']>=80].head(10)

,Media_Mf_overall,Numero_totale_match
Nome,,
Immobile,8.327143,221
Zapata D.,7.677143,218
Dybala,7.672857,201
Muriel,7.556667,168
Dzeko,7.482857,228
Martinez L.,7.475000,128
Caputo,7.465000,134
Belotti,7.440000,229
Mkhitaryan,7.380000,87


In [7]:
# Migliori fantamedie tra giocatori con più di due anni di storico in Serie A
mf_giocatori_under_80 = stats_serie_a[stats_serie_a['Almeno 15 pres ult stag']== 'SI'] \
.groupby('Nome') \
.agg(Media_Mf_overall=('Mf','mean'),Numero_totale_match = ('Pv','sum')) \
.sort_values(by='Media_Mf_overall', ascending=False)

mf_giocatori_over_80[mf_giocatori_over_80['Numero_totale_match']<80].head(10)

,Media_Mf_overall,Numero_totale_match
Nome,,
Ibrahimovic,8.000000,54
Osimhen,7.750000,51
Kean,7.685000,55
Abraham,7.620000,37
Arnautovic,7.470000,32
Giroud,7.330000,29
Beto,7.230000,28
Gonzalez N.,7.060000,33
Raspadori,7.056667,63


Vediamo quindi ora una sostanziale differenza tra le due tabelle, con la prima che identifica chi ha performato bene in un lungo arco temporale e la seconda che identifica chi ha performato bene nelle sue prime stagioni in serie A. Complessivamente i calciatori della prima tabella possono essere considerati delle assolute certezze (Immobile, Dybala, Martinez) mentre quelli della seconda tabella possono essere considerati degli interessanti prospetti (Raspadori, Beto, Abraham)

Fin qui è evidente che analizzando le fantamedie complessive, l'analisi si cocentri sugli attaccanti (che tipicamente hanno maggiore probabilità di portare bonus). Proviamo ora a replicare l'analisi anche per Portieri, Difensori e Centrocampisti

In [59]:
# Creiamo un dataset solo per i portieri, difensori e centrocampisti
stats_serie_a_por = stats_serie_a.query('R == "P"')
stats_serie_a_dif = stats_serie_a.query('R == "D"')
stats_serie_a_cen = stats_serie_a.query('R == "C"')
stats_serie_a_att = stats_serie_a.query('R == "A"')

# Salviamo i dataset nel folder
stats_serie_a_por.to_csv("por.csv", index=False)
stats_serie_a_dif.to_csv("dif.csv", index=False)
stats_serie_a_cen.to_csv("cen.csv", index=False)
stats_serie_a_att.to_csv("att.csv", index=False)


In [30]:
# Migliori fantamedie tra portieri con meno di due anni di storico in Serie A
mf_por_over_80 = stats_serie_a_por[stats_serie_a_por['Almeno 15 pres ult stag']== 'SI'] \
.groupby('Nome') \
.agg(Media_Mf_overall=('Mf','mean'),Numero_totale_match = ('Pv','sum')) \
.sort_values(by='Media_Mf_overall', ascending=False)

mf_por_over_80[mf_por_over_80['Numero_totale_match']>=80].head(10)

,Media_Mf_overall,Numero_totale_match
Nome,,
Szczesny,5.392857,209
Handanovic,5.304286,258
Cragno,4.944000,152
Skorupski,4.940000,206
Sirigu,4.854000,178
Musso,4.812500,135
Consigli,4.791429,252
Sepe,4.780000,124
Audero,4.724000,139


In [33]:
# Migliori fantamedie tra portieri con più di due anni di storico in Serie A
mf_por_under_80 = stats_serie_a_por[stats_serie_a_por['Almeno 15 pres ult stag']== 'SI'] \
.groupby('Nome') \
.agg(Media_Mf_overall=('Mf','mean'),Numero_totale_match = ('Pv','sum')) \
.sort_values(by='Media_Mf_overall', ascending=False)

mf_por_over_80[mf_por_over_80['Numero_totale_match']<80].head(10)

,Media_Mf_overall,Numero_totale_match
Nome,,
Maignan,5.640000,32
Terracciano,5.377500,53
Rui Patricio,5.040000,38
Vicario,4.860000,42
Montipo',4.385000,71
Milinkovic-Savic V.,4.367500,35
Provedel,4.116667,76


In [34]:
# Migliori fantamedie tra difensori con meno di due anni di storico in Serie A
mf_dif_over_80 = stats_serie_a_dif[stats_serie_a_dif['Almeno 15 pres ult stag']== 'SI'] \
.groupby('Nome') \
.agg(Media_Mf_overall=('Mf','mean'),Numero_totale_match = ('Pv','sum')) \
.sort_values(by='Media_Mf_overall', ascending=False)

mf_dif_over_80[mf_dif_over_80['Numero_totale_match']>=80].head(10)

,Media_Mf_overall,Numero_totale_match
Nome,,
Hernandez T.,6.716667,98
D'ambrosio,6.348571,155
Di Lorenzo,6.285000,139
Bonucci,6.275714,212
Acerbi,6.255714,246
Alex Sandro,6.235714,180
Rrahmani,6.216667,83
Dimarco,6.196000,88
Faraoni,6.196000,131


In [36]:
# Migliori fantamedie tra difensori con più di due anni di storico in Serie A
mf_dif_under_80 = stats_serie_a_dif[stats_serie_a_dif['Almeno 15 pres ult stag']== 'SI'] \
.groupby('Nome') \
.agg(Media_Mf_overall=('Mf','mean'),Numero_totale_match = ('Pv','sum')) \
.sort_values(by='Media_Mf_overall', ascending=False)

mf_dif_over_80[mf_dif_over_80['Numero_totale_match']<80].head(10)

,Media_Mf_overall,Numero_totale_match
Nome,,
Dumfries,6.780000,32
Cuadrado,6.610000,61
Singo,6.593333,65
Mari',6.470000,15
Demiral,6.385000,56
Smalling,6.343333,69
Tomori,6.340000,48
Darmian,6.326667,77
Lazzari,6.320000,28


In [39]:
# Migliori fantamedie tra centrocampisti con meno di due anni di storico in Serie A
mf_cen_over_80 = stats_serie_a_cen[stats_serie_a_cen['Almeno 15 pres ult stag']== 'SI'] \
.groupby('Nome') \
.agg(Media_Mf_overall=('Mf','mean'),Numero_totale_match = ('Pv','sum')) \
.sort_values(by='Media_Mf_overall', ascending=False)

mf_cen_over_80[mf_cen_over_80['Numero_totale_match']>=80].head(10)

,Media_Mf_overall,Numero_totale_match
Nome,,
Mkhitaryan,7.380000,87
Luis Alberto,7.058000,165
Malinovskyi,6.986667,94
Pasalic,6.880000,141
Verdi,6.878571,164
Calhanoglu,6.824000,167
Milinkovic-Savic,6.808571,229
Felipe Anderson,6.792500,129
Bonaventura,6.718571,181


In [42]:
# Migliori fantamedie tra centrocampisti con più di due anni di storico in Serie A
mf_cen_under_80 = stats_serie_a_cen[stats_serie_a_cen['Almeno 15 pres ult stag']== 'SI'] \
.groupby('Nome') \
.agg(Media_Mf_overall=('Mf','mean'),Numero_totale_match = ('Pv','sum')) \
.sort_values(by='Media_Mf_overall', ascending=False)

mf_cen_over_80[mf_cen_over_80['Numero_totale_match']<80].head(10)

,Media_Mf_overall,Numero_totale_match
Nome,,
Politano,7.010,36
Ribery,6.930,21
Bajrami,6.820,34
Orsolini,6.810,26
Miranchuk,6.695,34
Pobega,6.675,51
Zurkowski,6.620,33
Koopmeiners,6.590,28
Messias,6.560,26


In [44]:
# Per completezza replichiamo l'analisi sul dataset attaccanti anche se abbiamo già osservato i risultati
# Migliori fantamedie tra attaccanti con meno di due anni di storico in Serie A
mf_att_over_80 = stats_serie_a_att[stats_serie_a_att['Almeno 15 pres ult stag']== 'SI'] \
.groupby('Nome') \
.agg(Media_Mf_overall=('Mf','mean'),Numero_totale_match = ('Pv','sum')) \
.sort_values(by='Media_Mf_overall', ascending=False)

mf_att_over_80[mf_att_over_80['Numero_totale_match']>=80].head(10)

,Media_Mf_overall,Numero_totale_match
Nome,,
Immobile,8.327143,221
Zapata D.,7.677143,218
Dybala,7.672857,201
Muriel,7.556667,168
Dzeko,7.482857,228
Martinez L.,7.475000,128
Caputo,7.465000,134
Belotti,7.440000,229
Quagliarella,7.257143,229


In [45]:
# Migliori fantamedie tra attaccanti con più di due anni di storico in Serie A
mf_att_under_80 = stats_serie_a_att[stats_serie_a_att['Almeno 15 pres ult stag']== 'SI'] \
.groupby('Nome') \
.agg(Media_Mf_overall=('Mf','mean'),Numero_totale_match = ('Pv','sum')) \
.sort_values(by='Media_Mf_overall', ascending=False)

mf_att_over_80[mf_att_over_80['Numero_totale_match']<80].head(10)

,Media_Mf_overall,Numero_totale_match
Nome,,
Ibrahimovic,8.000000,54
Osimhen,7.750000,51
Kean,7.685000,55
Abraham,7.620000,37
Arnautovic,7.470000,32
Giroud,7.330000,29
Beto,7.230000,28
Gonzalez N.,7.060000,33
Raspadori,7.056667,63


### ANALISI SU TASSO DI CRESCITA DELLE FANTAMEDIE ###

A questo punto l'obiettivo è analizzare il tasso di crescita delle fantamedie per capire chi è in crescita stagione dopo stagione e chi invece è in una fase regressiva. L'obiettivo all'asta è acquistare un calciatore che sia in una fase crescente della sua carriera non calante

In [47]:
# Migliori crescta fantamedie tra portieri con meno di due anni di storico in Serie A
mf_por_over_80 = stats_serie_a_por[stats_serie_a_por['Almeno 15 pres ult stag']== 'SI'] \
.groupby('Nome') \
.agg(Media_Mf_overall=('Perc_Increase_Mf','mean'),Numero_totale_match = ('Pv','sum')) \
.sort_values(by='Media_Mf_overall', ascending=False)

mf_por_over_80[mf_por_over_80['Numero_totale_match']>=80].head(10)

,Media_Mf_overall,Numero_totale_match
Nome,,
Silvestri,0.206250,106
Skorupski,0.019853,206
Szczesny,0.009848,209
Musso,-0.002649,135
Handanovic,-0.004344,258
Cragno,-0.010601,152
Audero,-0.014891,139
Consigli,-0.018642,252
Sirigu,-0.025595,178


In [49]:
# Migliori fantamedie tra portieri con più di due anni di storico in Serie A
mf_por_under_80 = stats_serie_a_por[stats_serie_a_por['Almeno 15 pres ult stag']== 'SI'] \
.groupby('Nome') \
.agg(Media_Mf_overall=('Perc_Increase_Mf','mean'),Numero_totale_match = ('Pv','sum')) \
.sort_values(by='Media_Mf_overall', ascending=False)

mf_por_over_80[mf_por_over_80['Numero_totale_match']<80].head(10)

,Media_Mf_overall,Numero_totale_match
Nome,,
Montipo',0.197995,71
Provedel,0.127004,76
Milinkovic-Savic V.,0.064010,35
Terracciano,-0.010029,53
Vicario,-0.056000,42
Maignan,NaN,32
Rui Patricio,NaN,38


Vediamo dunque che in linea generale sono pochi i portieri che hanno migliorato le proprie prestazioni nel tempo. Abbiamo Silvestri, Skorupsky, Scezny, Montipò, Provedel e Milinkovic

In [50]:
# Migliori fantamedie tra difensori con meno di due anni di storico in Serie A
mf_dif_over_80 = stats_serie_a_dif[stats_serie_a_dif['Almeno 15 pres ult stag']== 'SI'] \
.groupby('Nome') \
.agg(Media_Mf_overall=('Perc_Increase_Mf','mean'),Numero_totale_match = ('Pv','sum')) \
.sort_values(by='Media_Mf_overall', ascending=False)

mf_dif_over_80[mf_dif_over_80['Numero_totale_match']>=80].head(10)

,Media_Mf_overall,Numero_totale_match
Nome,,
Bremer,0.064435,95
Skriniar,0.058039,209
Faraoni,0.055307,131
Rrahmani,0.046077,83
Calabria,0.029254,141
De Sciglio,0.027454,104
Dimarco,0.023509,88
Rodrigo Becao,0.023299,96
De Vrij,0.019723,187


In [51]:
# Migliori fantamedie tra difensori con più di due anni di storico in Serie A
mf_dif_under_80 = stats_serie_a_dif[stats_serie_a_dif['Almeno 15 pres ult stag']== 'SI'] \
.groupby('Nome') \
.agg(Media_Mf_overall=('Perc_Increase_Mf','mean'),Numero_totale_match = ('Pv','sum')) \
.sort_values(by='Media_Mf_overall', ascending=False)

mf_dif_over_80[mf_dif_over_80['Numero_totale_match']<80].head(10)

,Media_Mf_overall,Numero_totale_match
Nome,,
Udogie,0.101695,39
Ranieri L.,0.081238,30
Kalulu,0.059016,34
Buongiorno,0.055556,30
Carboni,0.053915,48
Darmian,0.035783,77
Cuadrado,0.028724,61
Bastoni S.,0.027375,50
Danilo,0.026440,77


In [52]:
# Migliori fantamedie tra centrocampisti con meno di due anni di storico in Serie A
mf_cen_over_80 = stats_serie_a_cen[stats_serie_a_cen['Almeno 15 pres ult stag']== 'SI'] \
.groupby('Nome') \
.agg(Media_Mf_overall=('Perc_Increase_Mf','mean'),Numero_totale_match = ('Pv','sum')) \
.sort_values(by='Media_Mf_overall', ascending=False)

mf_cen_over_80[mf_cen_over_80['Numero_totale_match']>=80].head(10)

,Media_Mf_overall,Numero_totale_match
Nome,,
Locatelli,0.068724,163
Tonali,0.055880,93
Milinkovic-Savic,0.044399,229
Barella,0.040972,194
Zaccagni,0.038535,104
Barak,0.036779,124
Pasalic,0.034765,141
Traore' Hj.,0.032941,118
Cristante,0.026806,187


In [53]:
# Migliori fantamedie tra centrocampisti con più di due anni di storico in Serie A
mf_cen_under_80 = stats_serie_a_cen[stats_serie_a_cen['Almeno 15 pres ult stag']== 'SI'] \
.groupby('Nome') \
.agg(Media_Mf_overall=('Perc_Increase_Mf','mean'),Numero_totale_match = ('Pv','sum')) \
.sort_values(by='Media_Mf_overall', ascending=False)

mf_cen_over_80[mf_cen_over_80['Numero_totale_match']<80].head(10)

,Media_Mf_overall,Numero_totale_match
Nome,,
Correa,0.127907,59
Politano,0.097027,36
Kastanos,0.071823,32
Sottil,0.059339,42
Schouten,0.047509,69
Makengo,0.045061,46
Haas,0.037037,22
Tameze,0.036848,71
Elmas,0.036611,72


In [54]:
# Migliori fantamedie tra attaccanti con meno di due anni di storico in Serie A
mf_att_over_80 = stats_serie_a_att[stats_serie_a_att['Almeno 15 pres ult stag']== 'SI'] \
.groupby('Nome') \
.agg(Media_Mf_overall=('Perc_Increase_Mf','mean'),Numero_totale_match = ('Pv','sum')) \
.sort_values(by='Media_Mf_overall', ascending=False)

mf_att_over_80[mf_att_over_80['Numero_totale_match']>=80].head(10)

,Media_Mf_overall,Numero_totale_match
Nome,,
Vlahovic,0.132768,103
Rafael Leao,0.073731,90
Immobile,0.054155,221
Martinez L.,0.051723,128
Orsolini,0.047129,111
Pinamonti,0.045401,97
Berardi,0.044563,208
Caprari,0.041236,178
Dzeko,0.036089,228


In [55]:
# Migliori fantamedie tra attaccanti con più di due anni di storico in Serie A
mf_att_under_80 = stats_serie_a_att[stats_serie_a_att['Almeno 15 pres ult stag']== 'SI'] \
.groupby('Nome') \
.agg(Media_Mf_overall=('Perc_Increase_Mf','mean'),Numero_totale_match = ('Pv','sum')) \
.sort_values(by='Media_Mf_overall', ascending=False)

mf_att_over_80[mf_att_over_80['Numero_totale_match']<80].head(10)

,Media_Mf_overall,Numero_totale_match
Nome,,
Pedro,0.111285,58
Bonazzoli,0.060735,71
Deulofeu,0.051732,63
Osimhen,0.050265,51
Sanabria,0.040268,76
Di Francesco F.,0.038072,41
Pussetto,0.010599,78
Raspadori,-0.023140,63
Vignato,-0.038760,25


### ANALISI GRAFICA ATTRAVERSO L'UTILIZZO DI MITO ###

In [62]:
# Creiamo prima un dataset per ruolo contenente solo giocatori con +15 presenze nella scorsa stagione
stats_serie_a_por_u = stats_serie_a_por[stats_serie_a_por['Almeno 15 pres ult stag']== 'SI']
stats_serie_a_dif_u = stats_serie_a_dif[stats_serie_a_dif['Almeno 15 pres ult stag']== 'SI']
stats_serie_a_cen_u = stats_serie_a_cen[stats_serie_a_cen['Almeno 15 pres ult stag']== 'SI']
stats_serie_a_att_u = stats_serie_a_att[stats_serie_a_att['Almeno 15 pres ult stag']== 'SI']

# Salviamo i dataset nel folder
stats_serie_a_por_u.to_csv("por_u.csv", index=False)
stats_serie_a_dif_u.to_csv("dif_u.csv", index=False)
stats_serie_a_cen_u.to_csv("cen_u.csv", index=False)
stats_serie_a_att_u.to_csv("att_u.csv", index=False)

In [56]:
import mitosheet

In [57]:
mitosheet.sheet(analysis_to_replay="id-izjykpnimu")

MitoWidget(analysis_data_json='{"analysisName": "id-gnjrwiqdxf", "analysisToReplay": {"analysisName": "id-izjy…

In [ ]:
from mitosheet import *; register_analysis("id-izjykpnimu");
    
# Imported Stats15_21 from Master_data_soccer_15_22.xlsx
import pandas as pd
sheet_df_dictonary = pd.read_excel(r'Master_data_soccer_15_22.xlsx', engine='openpyxl', sheet_name=['Stats15_21'], skiprows=0)
Stats15_21 = sheet_df_dictonary['Stats15_21']

# Imported dif.csv
import pandas as pd
dif = pd.read_csv(r'dif.csv')

# Imported dif.csv
import pandas as pd
dif_1 = pd.read_csv(r'dif.csv')

# Imported dif.csv
import pandas as pd
dif_2 = pd.read_csv(r'dif.csv')

# Deleted Stats15_21
del Stats15_21

# Deleted dif
del dif

# Deleted dif_1
del dif_1

# Deleted dif_2
del dif_2

# Imported dif_u.csv
import pandas as pd
dif_u = pd.read_csv(r'dif_u.csv')

# Pivoted into dif_u
tmp_df = dif_u[['Mf', 'Perc_Increase_Mf', 'R', 'Nome']]
pivot_table = tmp_df.pivot_table(
    index=['Nome', 'R'],
    values=['Mf', 'Perc_Increase_Mf'],
    aggfunc={'Mf': ['mean'], 'Perc_Increase_Mf': ['mean']}
)
pivot_table.set_axis([flatten_column_header(col) for col in pivot_table.keys()], axis=1, inplace=True)
dif_u_pivot = pivot_table.reset_index()

# Renamed dif_u to dif
dif = dif_u

# Renamed dif to dif_dataset
dif_dataset = dif

# Renamed dif_u_pivot to dif_pivot
dif_pivot = dif_u_pivot

# Imported cen_u.csv
import pandas as pd
cen_u = pd.read_csv(r'cen_u.csv')

# Renamed cen_u to cen_dataset
cen_dataset = cen_u

# Pivoted into cen_dataset
cen_dataset_pivot = pd.DataFrame(data={})

# Renamed cen_dataset_pivot to cen_pivot
cen_pivot = cen_dataset_pivot

# Deleted cen_pivot
del cen_pivot

# Pivoted into cen_dataset
tmp_df = cen_dataset[['Mf', 'Perc_Increase_Mf', 'R', 'Nome']]
pivot_table = tmp_df.pivot_table(
    index=['Nome', 'R'],
    values=['Mf', 'Perc_Increase_Mf'],
    aggfunc={'Mf': ['mean'], 'Perc_Increase_Mf': ['mean']}
)
pivot_table.set_axis([flatten_column_header(col) for col in pivot_table.keys()], axis=1, inplace=True)
cen_dataset_pivot = pivot_table.reset_index()

# Renamed cen_dataset_pivot to cen_pivot
cen_pivot = cen_dataset_pivot

# Imported att_u.csv
import pandas as pd
att_u = pd.read_csv(r'att_u.csv')

# Renamed att_u to att_dataset
att_dataset = att_u

# Pivoted into att_dataset
tmp_df = att_dataset[['Mf', 'Perc_Increase_Mf', 'R', 'Nome']]
pivot_table = tmp_df.pivot_table(
    index=['Nome', 'R'],
    values=['Mf', 'Perc_Increase_Mf'],
    aggfunc={'Mf': ['mean'], 'Perc_Increase_Mf': ['mean']}
)
pivot_table.set_axis([flatten_column_header(col) for col in pivot_table.keys()], axis=1, inplace=True)
att_dataset_pivot = pivot_table.reset_index()

# Renamed att_dataset_pivot to att_pivot
att_pivot = att_dataset_pivot

# Pivoted into dif_dataset
tmp_df = dif_dataset[['Squadra', 'Perc_Increase_Mf', 'R', 'Mf']]
pivot_table = tmp_df.pivot_table(
    index=['Squadra', 'R'],
    values=['Mf', 'Perc_Increase_Mf'],
    aggfunc={'Mf': ['mean'], 'Perc_Increase_Mf': ['mean']}
)
pivot_table.set_axis([flatten_column_header(col) for col in pivot_table.keys()], axis=1, inplace=True)
dif_dataset_pivot = pivot_table.reset_index()

# Sorted Mf mean in descending order
dif_dataset_pivot = dif_dataset_pivot.sort_values(by='Mf mean', ascending=False, na_position='last')

# Renamed dif_dataset_pivot to dif_sq_pivot
dif_sq_pivot = dif_dataset_pivot

# Pivoted into cen_dataset
tmp_df = cen_dataset[['Squadra', 'Perc_Increase_Mf', 'Mf']]
pivot_table = tmp_df.pivot_table(
    index=['Squadra'],
    values=['Mf', 'Perc_Increase_Mf'],
    aggfunc={'Mf': ['mean'], 'Perc_Increase_Mf': ['mean']}
)
pivot_table.set_axis([flatten_column_header(col) for col in pivot_table.keys()], axis=1, inplace=True)
cen_dataset_pivot = pivot_table.reset_index()

# Renamed cen_dataset_pivot to cen_sq_pivot
cen_sq_pivot = cen_dataset_pivot

# Pivoted into att_dataset
tmp_df = att_dataset[['Squadra', 'Perc_Increase_Mf', 'Mf']]
pivot_table = tmp_df.pivot_table(
    index=['Squadra'],
    values=['Mf', 'Perc_Increase_Mf'],
    aggfunc={'Mf': ['mean'], 'Perc_Increase_Mf': ['mean']}
)
pivot_table.set_axis([flatten_column_header(col) for col in pivot_table.keys()], axis=1, inplace=True)
att_dataset_pivot = pivot_table.reset_index()

# Renamed att_dataset_pivot to att_sq_pivot
att_sq_pivot = att_dataset_pivot

# Sorted Mf mean in descending order
cen_sq_pivot = cen_sq_pivot.sort_values(by='Mf mean', ascending=False, na_position='last')

# Sorted Mf mean in descending order
att_sq_pivot = att_sq_pivot.sort_values(by='Mf mean', ascending=False, na_position='last')
